## Model Implementation and validation Code:

### Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

## Performing Exploratory Analysis: 
It's important to explore the data and do basic analysis to understand the characteristic and form opinion / understanding of the data

### Data Structure:

In [ ]:
#Evaluating age distribution
plt.figure(figsize=(20,12))
#colors =["#774571","#b398af","#f1f1f1" ,"#afcdc7", "#6daa9f"]
Days_of_week=sns.countplot(x=data['age'],data=data, hue ="DEATH_EVENT",palette = cols)
Days_of_week.set_title("Distribution Of Age", color="#774571")

![image19.png](attachment:image19.png)</br>
### Outlier Analysis
Now we look for outliers using boxplot if any outlier found there should be remove that outlier.

In [ ]:
# Boxen and swarm plot of some non binary features.
feature = ["age","creatinine_phosphokinase","ejection_fraction","platelets","serum_creatinine","serum_sodium", "time"]
for i in feature:
    plt.figure(figsize=(8,8))
    sns.swarmplot(x=data["DEATH_EVENT"], y=data[i], color="black", alpha=0.5)
    sns.boxenplot(x=data["DEATH_EVENT"], y=data[i], palette=cols)
    plt.show()

: 

![image11.png](attachment:image11.png) ![image14.png](attachment:image14.png) ![image16.png](attachment:image16.png) ![image8.png](attachment:image8.png)</br>
<b>Outliers: Individual points beyond the interquartile range can be identified as outliers.</b>

In [ ]:
sns.kdeplot(x=data["age"], y=None, hue =data["DEATH_EVENT"], palette=cols)

: 

![image12.png](attachment:image12.png)</br>
- This KDE plot visualizes the distribution of death events, highlighting the age at which the majority of fatalities occurred or death happens. 

In [ ]:
data.info()

![image13.png](attachment:image13.png)</br>
There are no missing values.
### Correlation Matrix of the data

In [ ]:
#Examining a correlation matrix of all the features
cmap = sns.diverging_palette(275,150,  s=40, l=65, n=9)
corrmat = data.corr()
plt.subplots(figsize=(18,18))
sns.heatmap(corrmat,cmap= cmap,annot=True, square=True);

![image20.png](attachment:image20.png)
### Strong Positive Correlation

- Insight: Variables move in the same direction.

- Takeaway: Changes in one variable are likely to be associated with similar changes in the other.

### Strong Negative Correlation

- Insight: Variables move in opposite directions.

- Takeaway: Changes in one variable are likely to be associated with contrary changes in the other.

### Correlation Close to 1 or -1:


- Insight: Strong linear relationship.

- Takeaway: Changes in one variable can be predicted with a high degree of certainty based on the other.
### Notable points:
- Time of the patient's follow-up visit for the disease is crucial as initial diagnosis with cardiovascular issues and treatment reduces the chances of any fatality. It holds an inverse relation.
- Ejection fraction is the second most important feature. It is quite expected as it is basically the efficiency of the heart.
- Age of the patient is the third most correlated feature. Clearly as heart's functioning declines with aging
- With this bar plot identify the highest and lowest values. Compare the magnitudes of different categories. Look for patterns or trends.
- We understand with this that mostly 50 to 70 years old people mostly prone to heart diseas.